# 🤖 Robo Reviews

## Load all necessary packages

In [ ]:
# If you have any error remove the comment of the next line and install datasets
#%pip install datasets 
import pandas as pd
from utils.load_data import load_dataset_as_dataframe
from utils.data_cleaner import data_cleaner
from models.classifiers import rolland_model


[notice] A new release of pip available: 22.2.2 -> 25.3
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Load Data and Clean it up

In [ ]:
# 1. Load the Data
df1, df2, df3 = load_dataset_as_dataframe()

# 2. Clean and Merge the Data
dataframes_list = [df1, df2, df3]
source_names_list = ['1429_1', 'Datafiniti_Products', 'Datafinit_Products_May19']

# Clean the data and remove the duplicates
final_df = data_cleaner(dataframes_list, source_names_list)

# 3. Check the results
if not final_df.empty:
    print("\nFinal DataFrame Info")
    final_df.info()

    print("\nFinal DataFrame Head")
    display(final_df.head())
    
    print("\nData Source Distribution")
    display(final_df['data_source'].value_counts())
else:
    print("Data processing failed. Please check the logs.")

rolland_model(final_df)

1429_1.csv:   0%|          | 0.00/49.0M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Datafiniti_Amazon_Consumer_Reviews_of_Am(…):   0%|          | 0.00/99.6M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Datafiniti_Amazon_Consumer_Reviews_of_Am(…):   0%|          | 0.00/266M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Processing DataFrame from '1429_1'...
Finished processing '1429_1'. Found 34660 rows.
Processing DataFrame from 'Datafiniti_Products'...
Finished processing 'Datafiniti_Products'. Found 5000 rows.
Processing DataFrame from 'Datafinit_Products_May19'...
Finished processing 'Datafinit_Products_May19'. Found 28332 rows.

Merging all datasets...
Total rows after merge: 67992
Dropped 2153 duplicate reviews.
Dropped 6810 rows due to missing (MUST HAVE) data.
Final merged dataset has 59029 rows.

Final DataFrame Info
<class 'pandas.core.frame.DataFrame'>
Index: 59029 entries, 0 to 67991
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   product_name          59029 non-null  object 
 1   categories            59029 non-null  object 
 2   review_rating         59029 non-null  float64
 3   review_text           59029 non-null  object 
 4   review_title          59029 non-null  object 
 5   image_urls        

,product_name,categories,review_rating,review_text,review_title,image_urls,review_date,review_recommend,review_helpful_count,source_urls,review_username,data_source
0,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...","Electronics,iPad & Tablets,All Tablets,Fire Ta...",5.0,This product so far has not disappointed. My c...,Kindle,NaN,2017-01-13T00:00:00.000Z,True,0.0,NaN,Adapter,1429_1
1,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...","Electronics,iPad & Tablets,All Tablets,Fire Ta...",5.0,great for beginner or experienced person. Boug...,very fast,NaN,2017-01-13T00:00:00.000Z,True,0.0,NaN,truman,1429_1
2,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...","Electronics,iPad & Tablets,All Tablets,Fire Ta...",5.0,Inexpensive tablet for him to use and learn on...,Beginner tablet for our 9 year old son.,NaN,2017-01-13T00:00:00.000Z,True,0.0,NaN,DaveZ,1429_1
3,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...","Electronics,iPad & Tablets,All Tablets,Fire Ta...",4.0,I've had my Fire HD 8 two weeks now and I love...,Good!!!,NaN,2017-01-13T00:00:00.000Z,True,0.0,NaN,Shacks,1429_1
4,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...","Electronics,iPad & Tablets,All Tablets,Fire Ta...",5.0,I bought this for my grand daughter when she c...,Fantastic Tablet for kids,NaN,2017-01-12T00:00:00.000Z,True,0.0,NaN,explore42,1429_1



Data Source Distribution


data_source
1429_1                      27863
Datafinit_Products_May19    26274
Datafiniti_Products          4892
Name: count, dtype: int64